# Getting available actions
Actions are configured in `app\configurations` directory. When the UI makes a request to get the actions available for an object, the `app/ajaxviews/actions.py` loads the yaml, and interprets what actions that the selected object is available to take. Those actions are sent to the user. 

### Two times when actions come up:
* web interaction that creates jobs
* azure functions that resolve actions

In [2]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append("../..")
sys.path.append("..")

import yaml, ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import sys, os

from app.connectors import cmdb_graph
from app.objects import time as t

import app.ajaxviews.actions as actions
import app.models as models

c = models.CosmosdbClient()

from helpers import test_queries

executing local windows deployment


## web interaction that creates jobs

`app/ajaxviews/actions.py` takes input from the browser and creates jobs. Tools exist in that library to:
* validate which jobs an agent can do
* build the job and action graph elelements
* update the graph with the job

the resulting addition will be some kind of variant of:
`(action)<-job-(agent)`


In [3]:
# The pop here is our agent. We'll just grab one randomly. 
pop = test_queries.get_random_pop(c,'BillmanLocal2')
pop


{'isIdle': 'true',
 'name': 'Obattla Artestai',
 'objid': '3735917286448',
 'conformity': 0.416,
 'literacy': 0.332,
 'aggression': 0.629,
 'constitution': 0.654,
 'health': 0.42,
 'isIn': '8822366435138',
 'industry': 0.343,
 'wealth': 0.286,
 'factionLoyalty': 0.463,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '3735917286448'}

You can load the full actions list. The actions is kept in a Yaml file. This comes from `app/configurations`. It's only used in the Ajax views as the action is stored in the graph, so no need to copy to the funciton. 

In [4]:
all_actions = actions.get_actions_config()
pd.DataFrame(all_actions)

,type,label,applies_to,effort,requires_attr,augments_self_properties,comment
0,healthcare_initiatives,action,pop,2,{'wealth': 0.1},"{'health': 0.05, 'wealth': '-.05'}",increase the health of your population slightl...
1,individual_education,action,pop,2,{'wealth': 0.1},"{'wealth': '-.1', 'literacy': 0.1, 'factionLoy...","expand privatized education, increase literacy..."
2,patriot_education,action,pop,1,{'factionLoyalty': 0.1},"{'factionLoyalty': 0.05, 'literacy': 0.1, 'agg...",expand public education programs with a collec...
3,patriot_propoganda,action,pop,1,NaN,"{'factionLoyalty': 0.1, 'conformity': 0.1, 'ag...","build national pride, increasin faction loyalt..."
4,build_infrastructure,action,pop,3,{'wealth': 0.5},"{'wealth': -0.1, 'industry': 0.1}",increase industry by investing in local infras...
5,build_building,action,pop,1,NaN,NaN,opens the building window for that population ...


`validator.validate()` returns the shotened list. of the actions that the population (or agent) can take. 

In [5]:
validator = actions.ActionValidator(pop, all_actions)
pd.DataFrame(validator.validate())

,type,label,applies_to,effort,requires_attr,augments_self_properties,comment
0,healthcare_initiatives,action,pop,2,{'wealth': 0.1},"{'health': 0.05, 'wealth': '-.05'}",increase the health of your population slightl...
1,individual_education,action,pop,2,{'wealth': 0.1},"{'wealth': '-.1', 'literacy': 0.1, 'factionLoy...","expand privatized education, increase literacy..."
2,patriot_education,action,pop,1,{'factionLoyalty': 0.1},"{'factionLoyalty': 0.05, 'literacy': 0.1, 'agg...",expand public education programs with a collec...
3,patriot_propoganda,action,pop,1,NaN,"{'factionLoyalty': 0.1, 'conformity': 0.1, 'ag...","build national pride, increasin faction loyalt..."
4,build_building,action,pop,1,NaN,NaN,opens the building window for that population ...


## Creating a job

to create a job you need an agent and and action. This is done in the ajax views for that class (e.g. `app/ajaxviews/pop`) because they are a request from the UI.



In [6]:
def create_job(pop, action, universalTime):
    if type(universalTime) == list:
        universalTime = universalTime[0]
    time_to_complete = int(universalTime["currentTime"]) + int(action["effort"])
    # not actually creating the node here, just demo.
    # uid = create_action_node(c,action,pop)
    popToAction = {
        "node1": pop["objid"],
        "node2": "1234567890",
        "label": "takingAction",
        "name": "takingAction",
        "weight": time_to_complete,
        "actionType": action["type"],
        "status": "pending",
    }
    edges = [popToAction]
    return edges


time = {"currentTime": 10000}
create_job(pop, all_actions[0], time)

[{'node1': '3735917286448',
  'node2': '1234567890',
  'label': 'takingAction',
  'name': 'takingAction',
  'weight': 10002,
  'actionType': 'healthcare_initiatives',
  'status': 'pending'}]

In [7]:
time = t.Time(c)

In [8]:
time.get_global_actions()
time.actions

[]

All actions require both an agent, job and action. The properties of the action determine how it affects other things. 


### Augments self properties
Takes a dictionary of properties and augments them. Adds the property if it does not exist. 
